In [57]:
from __future__ import print_function, division
import csv
import argparse
import os
import math
import re
from PyPDF2 import PdfFileMerger
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from random import randint, choice, uniform
import cv2
import sys
from std_msgs.msg import String

apriltag_family_choice = "tag36h11"
apriltag_family = ["tag16h5",
                   "tag36h11",
                   "tag25h9",
                   "tagCircle21h7",
                   "tagCircle49h12",
                   "tagCustom48h12",
                   "tagStandard41h12",
                   "tagStandard52h13"]
if apriltag_family_choice not in apriltag_family:
    print('{:s} is an invalid AprilTag family, please use one of: {:s}'.format(apriltag_family_choice, apriltag_family))
else:
    number_of_apriltags_in_bundle = 2
    bundle_label_size_inches_horizontal_vertical = [3.5,8.5]
    orientation_indicator_type = "id-lower"
    auto_scale_placement = True
    create_bundle_yaml = True
    number_of_bundles = 20
    no_repeat_tags_in_bundle = True
    border_offset_to_label_size_ratio = .15
    resources_dir = os.path.realpath('../resources')
    font_file = os.path.join(resources_dir, 'OpenSans-CondLight.ttf')
    save_bundle_images = True
    save_bundle_images_folder = '/tmp/{:d}x{:s}'.format(int(number_of_apriltags_in_bundle), apriltag_family_choice)
    
    if save_bundle_images:
        try:  
            os.mkdir(save_bundle_images_folder)
        except OSError:  
            print ('Creation of the directory {:s} failed, please check that it does not already exist.'.format(save_bundle_images_folder))
        else:
            print ('Successfully created the directory {:s}'.format(save_bundle_images_folder))

            
def gen_id_combinations():
    label_dict = [{}]
    wlbl, hlbl = bundle_label_size_inches_horizontal_vertical
    if float(w)/float(h) < .5:
        aligned="vertical"
    elif float(w)/float(h) > 2:
        aligned="horizontal"
        print("Horizontally aligned label is not yet enabled")
        return
    else:
        print("Invalid label aspect ratio, try label with a width-height or height-width ratio of greater than 2")
        return
def gen_bundle_imgs():
    dpi = 200
    hbgn=int(bundle_label_size_inches_horizontal_vertical[1]*dpi)
    wbgn=int(bundle_label_size_inches_horizontal_vertical[0]*dpi)
    bg_bc = Image.new(mode='RGB', size=(wbgn,hbgn), color=(255,255,255)).convert('RGBA')
    bg_bc_d = ImageDraw.Draw(bg_bc)
    font_size = min(bundle_label_size_inches_horizontal_vertical)/10.0*dpi
    font = ImageFont.truetype(font_file, int(font_size))
    bg_bc_d.text(list(lbl_str_l.astype(np.int32)),lbl_pr,(0,0,0))

    data_lbl=code128_format(lbl)
    img_bc_lbl=code128_image(data_lbl)
    
    bc_p=.9025
    img_bc_lbl = img_bc_lbl.crop((int((img_bc_lbl.size[0]/2)*(1-bc_p)),0,int((img_bc_lbl.size[0]/2)*(1+bc_p)),img_bc_lbl.size[1]))
    img_bc_lbl = img_bc_lbl.resize(list(lbl_bc_wh.astype(np.int32)), Image.ANTIALIAS)
    bg_bc.paste(img_bc_lbl,list(lbl_bc_l.astype(np.int32)))
    
    return (bg_bc)

def create_labels(file_name_csv, dpi):
    label_file = open(file_name_csv, 'rb')
    csvfile=csv.reader(label_file)
    pw, ph = (int(8.5*dpi), int(11.0*dpi))
    pdf_image_path = '/tmp/label_rk.png'
    pdf_rk_tmp = Image.new("RGB",(pw, ph),"white")
    pdf_rk_tmp.save(pdf_image_path)

    lbl_w_p = 4.0 #in
    lbl_h_p = 4.0 #in

    c = [(int(((3.0/16.0)+(1.0*lbl_w_p/2.0))*dpi),int(((7.0/16.0)+(1.0*lbl_h_p/2.0))*dpi)),
         (int((8.5-(3.0/16.0)-(1.0*lbl_w_p/2.0))*dpi),int(((7.0/16.0)+(1.0*lbl_h_p/2.0))*dpi)),
         (int((8.5-(3.0/16.0)-(1.0*lbl_w_p/2.0))*dpi),int((11.0-(7.0/16.0)-(1.0*lbl_h_p/2.0))*dpi)),
         (int(((3.0/16.0)+(1.0*lbl_w_p/2.0))*dpi),int((11.0-(7.0/16.0)-(1.0*lbl_h_p/2.0))*dpi))]
    
    label_list = []
    for row in csvfile:
        for col in row:
            label_list.append(col)
    max_pages = str(int(math.ceil(float(len(label_list))/float(len(c)))))
    pdf_list=list()
    pg=0
    pdf_page = Image.open(pdf_image_path)
    pdf_page=pdf_page.convert('RGB').convert('RGBA')
    lbl_pg_cnt = 0
    for slot in label_list:
        bg_bc = gen_bar(slot, dpi)
        coord=(int(c[lbl_pg_cnt][0]-dpi*(lbl_w_p/2.0)),int(c[lbl_pg_cnt][1]-dpi*(lbl_h_p/2.0)))
        pdf_page.alpha_composite(bg_bc,coord)
        lbl_pg_cnt += 1
        if lbl_pg_cnt == len(c):
            lbl_pg_cnt = 0
            pdf_page=pdf_page.convert('RGB').convert('L')
            save_str='/tmp/LBL_'+str(pg)+'.pdf'
            pdf_page.save(save_str, "PDF", resolution=600)
            print('Page: '+str(pg+1)+'/'+max_pages)
            pdf_list.append(save_str)
            pg+=1
            pdf_page = Image.open(pdf_image_path)
            pdf_page=pdf_page.convert('RGB').convert('RGBA')
        elif slot == label_list[-1]:
            pdf_page=pdf_page.convert('RGB').convert('L')
            save_str='/tmp/LBL_'+str(pg)+'.pdf'
            pdf_page.save(save_str, "PDF", resolution=600)
            print('Page: '+str(pg+1)+'/'+max_pages)
            pdf_list.append(save_str)

    print('Merging PDF files')
    merger = PdfFileMerger()
    for pdf in pdf_list:
        merger.append(pdf)
    merger.write("/tmp/LBS.pdf")
    print('Finished merging')
    return(True)
#create_labels(file_name_csv, ppi)



Successfully created the directory /tmp/2xtag36h11


In [62]:
import numpy as np
aligned="vertical"
number_of_bundles = 20
Unique_Bundles=[]
number_of_family_labels = len(next(os.walk(os.path.realpath('../apriltag-imgs/{:s}'.format(apriltag_family_choice))))[2])
first_tag_set=np.array(range(number_of_family_labels))
second_tag_set=np.array(range(number_of_family_labels))
for i in range(len(first_tag_set)):
    second_tag_set=np.delete(second_tag_set,0)
    for k in range(len(second_tag_set)):
        Unique_Bundles.append([first_tag_set[i],second_tag_set[k]])
        if number_of_bundles == len(Unique_Bundles):
            break
    if number_of_bundles == len(Unique_Bundles):
        break
if len(Unique_Bundles) < number_of_bundles:
    print('Requested {:d} bundels but family {:s} only supports up to {:d} unique bundles.'.format(number_of_bundles,apriltag_family_choice,len(Unique_Bundles)))
print('Proceeding to generate {:d} bundles with {:d} {:s} aligned tags of {:s} family.'.format(len(Unique_Bundles), number_of_apriltags_in_bundle, aligned, apriltag_family_choice))

Proceeding to generate 20 bundles with 2 vertical aligned tags of tag36h11 family.
